1\)

Importing Data

In [1]:
import numpy as np

from sklearn import datasets
iris = datasets.load_iris()
X = iris.data[:,0:4] ## from this only take features 0,1,2,3
Y = iris.target


Naive Bayes Code

In [2]:
def PGauss(mu, sig, x):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.) + 1e-300) )

def processing_for_NB(X, Y):
    U = np.zeros((X.shape[1], 3))
    S = np.zeros((X.shape[1], 3))
    PClass = []

    for c in range(3):
        PClass.append(np.array(np.where(Y==c)).shape[1]/Y.shape[0])
        for f in range(X.shape[1]):
            U[f,c] = X[np.where(Y==c),f].mean()
            S[f,c] = X[np.where(Y==c),f].std()

    return PClass, U, S

def NB(X, PClass, U, S, num_classes = 3):
    Y = np.ones((X.shape[0], num_classes))
    for i in range(Y.shape[0]): #0-149
        for c in range(Y.shape[1]): #0-3
            Y[i,c] *= PClass[c]
            for f in range(X.shape[1]): #0-4
                Y[i,c] *= PGauss(U[f,c], S[f,c], X[i, f])
    return Y

def argmax(Y):
    for i in range(Y.shape[0]):
        maxz = max(Y[i])
        for j in range(len(Y[i])):
            Y[i,j] = 1.0 if Y[i,j]>=maxz else 0
    return Y


In [3]:
from sklearn.preprocessing import OneHotEncoder

def onehot(Y):
    onehot_encoder = OneHotEncoder(sparse_output=False)
    integer_encoded = Y.reshape(len(Y), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return(onehot_encoded)

2\) Testing Naive Bayes's Accuracy

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test_ = train_test_split(X, Y, test_size=0.3)

PClass, U, S = processing_for_NB(X_train, Y_train)

Y_pred = NB(X_test, PClass, U, S)
Y_pred = argmax(Y_pred)

Y_test = onehot(Y_test_)

def accuracy(Y, Y_pred):
    correct_pred = 0
    misclassifications = 0
    idx = []
    i = 0
    for y, y_pred in zip(Y, Y_pred):
        if np.all(y == y_pred):
            correct_pred += 1
        else:
            misclassifications +=1
            idx.append(i)
        i += 1
    return correct_pred/Y.shape[0], misclassifications, idx

correct_pred, misclassifications, err_idx = accuracy(Y_test, Y_pred)
print("Training Samples: ", X_train.shape[0], "Testing Samples: ", X_test.shape[0])
print("Accuracy: ", correct_pred, "Misclassifications: ", misclassifications) 
print("Misclassified Indexes: ", err_idx, "Actual Classification: ", Y_test[err_idx], "NB Prediction: ", Y_pred[err_idx] )

Training Samples:  105 Testing Samples:  45
Accuracy:  0.9777777777777777 Misclassifications:  1
Misclassified Indexes:  [16] Actual Classification:  [[0. 1. 0.]] NB Prediction:  [[0. 0. 1.]]


3\) Sklearn Implementation

In [14]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

clf = GaussianNB()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
print("Accuracy: ", accuracy_score(Y_test_, Y_pred))


Accuracy:  1.0


Comparision: The canned sklearn model works very well and gets 0 misclassifications however, my implementation of Naive Bayes gets 1 misclassification.